In [1]:
import requests
from bs4 import BeautifulSoup
import ollama
import json

In [2]:
url = "https://www.nationalgeographic.com.es/ciencia/porfiria-enfermedad-metabolica-dio-lugar-vampiros_26542"

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
response = requests.get(url, headers=headers)
    

soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
title = soup.find("h1", class_="article-title").text.strip()

In [5]:
print(title)

Porfiria, la enfermedad metabólica que dio lugar a los vampiros


In [6]:
paragraphs = soup.find_all('div', class_='article-text txt')

In [13]:
text = ""
for i in paragraphs[0].find_all("p"):
    if not i.attrs:
        print(i.text)
        text += i.text + "\n"

A lo largo de la historia de la humanidad, los vampiros han sido objeto de fascinación, de miedo y mitos. Hoy en día, la leyenda sobre la crueldad del Conde Drácula, así como otras criaturas nocturnas han hecho mella en el imaginario popular, aunque sus relatos se han ido suavizando desde finales del siglo XIX hasta la fecha. Con el paso de los años, las habilidades de los vampiros han ido variando, pero por lo general siempre han sido criaturas esquivas, a los que no les puede dar el sol y a los que se le combate con plata, ajos, y, para darles la estocada final, se necesita una estaca de madera.
Si me invitan a pasar a través de sus pantallas, podrán leer la historia de cómo las leyendas se apoyan en fenómenos otrora inexplicables, pero que en la actualidad su halo de misticismo ha ido desvaneciéndose gracias a la ciencia. En el caso de los vampiros, estos seres podrían haber surgido a partir de enfermedades metabólicas raras llamadas porfirias, cuyos síntomas se asemejan vagamente a

In [14]:
prompt = f"""
    Basado en el siguiente artículo con título "{title}", realiza dos tareas y devuelve el resultado en formato JSON:
    Formato de salida esperado:
    {{"resumen": "...","etiquetas": ["...", "...", "..."]}}
    1. Genera un "resumen"  conciso en español que no exceda los 200 caracteres.
    2. Genera "etiquetas", una lista de exactamente tres keywords en español.

    Cuerpo del artículo:
    {text}
    """
response = ollama.generate(
            model='llama3.2:latest',
            prompt=prompt,
            format='json',
            stream=False
        )

In [15]:
content = response['response']

In [16]:
data = json.loads(content)
summary = data.get("resumen", "No se pudo generar el resumen.")
tags = data.get("etiquetas", [])

In [17]:
print(data)

{'resumen': 'La porfiria es una enfermedad metabólica que puede causar síntomas similares a los de los vampiros, como sensibilidad al sol y lesiones en la piel. La ciencia ha avanzado en su diagnóstico y tratamiento, pero muchos pacientes siguen enfrentando desafíos reales.', 'etiquetas': ['Porfiria', 'Enfermedad metabólica', 'Vampiros']}


In [18]:
print(summary)

La porfiria es una enfermedad metabólica que puede causar síntomas similares a los de los vampiros, como sensibilidad al sol y lesiones en la piel. La ciencia ha avanzado en su diagnóstico y tratamiento, pero muchos pacientes siguen enfrentando desafíos reales.


In [19]:
print(tags)

['Porfiria', 'Enfermedad metabólica', 'Vampiros']
